In [1]:
# import sys
# import gymnasium as gym
# sys.modules["gym"] = gym
import gym
gym.__version__

'0.21.0'

In [2]:
import os
import numpy as np
np.random.seed(0)
import torch
print(torch.cuda.is_available())

True


In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback

In [4]:
env_id = "MountainCarContinuous-v0"
NUM_CPU = 32  # Number of processes to use

In [7]:
# Parallel environments
train_env_f = make_vec_env(env_id, n_envs=NUM_CPU)
train_env_f.observation_space

/home/vlad1slav/Diploma/rl-venv/lib/python3.10/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Box([-1.2  -0.07 -3.  ], [0.6  0.07 3.  ], (3,), float32)

In [8]:
# for i in range(10):
for i in np.random.randint(2**16, size=10):
    model = PPO("MlpPolicy", 
            train_env_f,
            verbose=1, 
            seed=int(i),
            batch_size=512,
            learning_rate=7.77e-05,
            n_epochs=10,
            n_steps=8*NUM_CPU,
            gae_lambda=0.9,
            gamma=0.9999,
            max_grad_norm=5,
            use_sde=True,
            tensorboard_log="./logs/ppo_MountainCar_tensorboard_expert/")
    
    eval_callback = EvalCallback(train_env_f, 
                                 best_model_save_path=f"./best_models/expert{i}",
                                 log_path=f"./results/expert{i}",
                                 eval_freq=2**13/NUM_CPU,
                                 deterministic=True, render=False)

    model.learn(total_timesteps=1_000_000, callback=eval_callback, progress_bar=True)
    model.save(f"./models/expert{i}")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1,007,616/1,000,000  [ 0:01:22 < 0:00:00 , 15,143 it/s ]
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1,007,616/1,000,000  [ 0:01:22 < 0:00:00 , 15,143 it/s ]

In [9]:
train_env = make_vec_env(env_id, 
                         env_kwargs=dict(
                            full_obs=False
                         ),
                         n_envs=NUM_CPU)
train_env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [10]:
for i in range(10):
# for i in np.random.randint(2**16, size=10):
    model = PPO("MlpPolicy", 
            train_env,
            verbose=1, 
            seed=int(i),
            batch_size=512,
            learning_rate=7.77e-05,
            n_epochs=10,
            n_steps=8*NUM_CPU,
            gae_lambda=0.9,
            gamma=0.9999,
            max_grad_norm=5,
            use_sde=True,
            tensorboard_log="./logs/ppo_MountainCar_tensorboard_noob/")
    
    eval_callback = EvalCallback(train_env, 
                                 best_model_save_path=f"./best_models/noob{i}",
                                 log_path=f"./results/noob{i}",
                                 eval_freq=2**13/NUM_CPU,
                                 deterministic=True, render=False)

    model.learn(total_timesteps=1_000_000, callback=eval_callback, progress_bar=True)
    model.save(f"./models/noob{i}")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1,007,616/1,000,000  [ 0:01:24 < 0:00:00 , 14,503 it/s ]
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1,007,616/1,000,000  [ 0:01:24 < 0:00:00 , 14,503 it/s ]